In [101]:
from lark import Lark, Transformer

mu_parser = Lark.open("grammars/mcf.lark", parser="earley")
formula = mu_parser.parse(open("../examples/test.mu").read())
print(formula.pretty())

start
  formula
    nu
      X
      formula
        mu
          Y
          formula
            lor
              formula
                diamond
                  plato
                  formula	Y
              formula
                diamond
                  others
                  formula	X



In [102]:
class TreeToJson(Transformer):
    def start(self, f):
        (f,) = f
        return f
    def formula(self, f):
        (f,) = f
        return f
    
    TRUE = lambda self, _: {"val": True}
    FALSE = lambda self, _: {"val": False}
    def VAR(self, f):
        f = str(f)
        return {"var": f}
    def AL(self, f):
        f = str(f)
        return f
    def land(self, f):
        return {"and": f}
    def lor(self, f):
        return {"or": f}
    def diamond(self, f):
        (a, f) = f
        return {"diamond": [a, f]}
    def box(self, f):
        (a, f) = f
        return {"box": [a, f]}
    def mu(self, f):
        return {"mu": f}
    def nu(self, f):
        return {"nu": f}

print(TreeToJson().transform(formula))

{'nu': [{'var': 'X'}, {'mu': [{'var': 'Y'}, {'or': [{'diamond': ['plato', {'var': 'Y'}]}, {'diamond': ['others', {'var': 'X'}]}]}]}]}


In [103]:
def reduceFormula(t, reduceFixPoints=True):
    operand = list(t.keys())[0]
    arguments = t[operand]

    def negate(t, x):
        o = list(t.keys())[0]
        a = t[o]

        if o == "var":
            if a == x["var"]:
                return {"neg": t}
            else:
                return t
        elif o == "val":
            return t
        elif o == "and" or o == "or":
            return {o: [negate(arg, x) for arg in a]}
        elif o == "box" or o == "diamond":
            return {o: [a[0], negate(a[1], x)]}
        elif o == "mu" or o == "nu":
            v = arguments[0]
            f = arguments[1]
            return {o: [v, negate(f, x)]}
        else:
            return {o: negate(a, x)}

    if operand == "val":
        if arguments == False:
            return {"neg": {"val": True}}
        else:
            return t
    elif operand == "var":
        return t
    elif operand == "diamond":
        return {"neg": {"box": [arguments[0], {"neg": reduceFormula(arguments[1], reduceFixPoints)}]}}
    elif operand == "box":
        return {"box": [arguments[0], reduceFormula(arguments[1], reduceFixPoints)]}
    elif operand == "and" or operand == "or":
        return {operand: [reduceFormula(arg, reduceFixPoints) for arg in arguments]}
    elif operand == "mu":
        v = arguments[0]
        f = arguments[1]
        if reduceFixPoints:
            return {"neg": {"nu": [v, reduceFormula(negate(f, v), reduceFixPoints)]}}
        else:
            return {operand: [v, reduceFormula(f, reduceFixPoints)]}
    elif operand == "nu":
        v = arguments[0]
        f = arguments[1]
        return {operand: [v, reduceFormula(f, reduceFixPoints)]}
    else:
        return {operand: reduceFormula(arguments, reduceFixPoints)}


In [104]:
%store -r lts
counter = 0
#use a dictoinary to keep track of the variables
variables = dict()  
# run solver with a lts a formula and the type of algorithm desired, True for naive
# false for emmerson-lei
def solver(lts,formula,naive=True):
    global counter
    counter = 0
    if naive:
        states = simple(lts,reduceFormula(TreeToJson().transform(formula)))
    else:
        states = emmerson(lts,reduceFormula(TreeToJson().transform(formula),False))
    print(0 in states)
    print(counter)
    #temp
    print(states)
    

def simple(lts, formula):
    global counter
    states = lts.states
    operand = list(formula.keys())[0]
    arguments = formula[operand]
    if operand == "neg":
        return states - (simple(lts,arguments))
    elif operand == "or":
        return simple(lts,arguments[0]) | simple(lts,arguments[1])
    elif operand == "and":
        return simple(lts,arguments[0]) & simple(lts,arguments[1])
    elif operand == "box":
        return lts.box(simple(lts,arguments[1]),arguments[0])
    elif operand == "val":
        return states
    elif operand == "var":
        return variables[arguments] 
    elif operand == "nu":
        variable = arguments[0]["var"]
        #we start with all the states
        if (type != "nu" or (not variable in variables)):
            variables[variable] = states
        counter = counter +1
        newSol = emmerson(lts,arguments[1],"nu")
        while newSol != variables[variable]:
            counter += 1
            variables[variable] = newSol 
            newSol = emmerson(lts,arguments[1],"nu")
        return newSol
    else:
        return states

def emmerson(lts, formula,type = "null"):
    
    global counter
    states = lts.states
    operand = list(formula.keys())[0]
    arguments = formula[operand]
    if operand == "neg":
        return states - (emmerson(lts,arguments,type))
    elif operand == "or":
        return emmerson(lts,arguments[0],type) | emmerson(lts,arguments[1],type)
    elif operand == "and":
        return emmerson(lts,arguments[0],type) & emmerson(lts,arguments[1],type)
    elif operand == "box":
        return lts.box(emmerson(lts,arguments[1],type),arguments[0])
    elif operand == "val":
        return states
    elif operand == "var":
        return variables[arguments] 
    elif operand == "nu":
        variable = arguments[0]["var"]
        #we start with all the states
        if (type != "nu" or (not variable in variables)):
            variables[variable] = states
        counter = counter +1
        newSol = emmerson(lts,arguments[1],"nu")
        while newSol != variables[variable]:
            counter += 1
            variables[variable] = newSol 
            newSol = emmerson(lts,arguments[1],"nu")
            print("nu")
            print(newSol)
        return newSol
    elif operand == "mu":
        variable = arguments[0]["var"]
        #we start with all the states
        
        if (type != "mu" or (not variable in variables)):
            variables[variable] = set()
        counter = counter +1
        newSol = emmerson(lts,arguments[1],"mu")
        while newSol != variables[variable]:
            print(variables[variable])
            counter += 1
            variables[variable] = newSol 
            newSol = emmerson(lts,arguments[1],"mu")
            print("mu")
            
        return newSol
    else:
        return states








solver(lts,formula,False)





set()
mu
set()
mu
nu
{2051, 2052, 3078, 2056, 1034, 3085, 2083, 3109, 3111, 3113, 3114, 3116, 3117, 1074, 1079, 1080, 3141, 3148, 2132, 2136, 2139, 2140, 3166, 2144, 2150, 2151, 3174, 2154, 3191, 3193, 3194, 123, 127, 130, 132, 3204, 3210, 2192, 1173, 3221, 2215, 1193, 1198, 1199, 3251, 3255, 1209, 188, 1213, 1212, 1214, 192, 1215, 1216, 195, 1225, 2259, 2261, 2266, 2267, 3298, 3302, 232, 3305, 235, 257, 1301, 3350, 3352, 2334, 2336, 2343, 296, 2344, 299, 301, 3373, 303, 304, 305, 2351, 3375, 308, 310, 312, 313, 315, 1340, 317, 2364, 1343, 1345, 2380, 3406, 3408, 1365, 1367, 3420, 353, 1377, 356, 2405, 3428, 3435, 3436, 1391, 378, 1405, 2430, 1407, 2434, 3459, 2437, 1427, 1428, 1431, 3479, 1434, 1436, 1438, 1440, 417, 1442, 1443, 420, 1444, 1445, 424, 425, 426, 1448, 1450, 429, 1452, 1453, 433, 434, 438, 3510, 2498, 2503, 2504, 459, 3533, 491, 3566, 495, 496, 500, 1531, 1534, 2573, 527, 2580, 2581, 1560, 2588, 1568, 1580, 1583, 1605, 583, 585, 587, 588, 590, 592, 598, 599, 600, 601, 60